declarations

set variables - server, table, columns (could be tidied later with analysis being loaded), key field

set dict of hub

get from db for types etc for sat

append to static fields for sat

In [150]:
import json
import os
import sqlite3

from pathlib import Path
from datetime import datetime

conn = sqlite3.connect('../full_metadata.db')
cur = conn.cursor()

time_string = datetime.now().strftime('%Y%m%d%H%M%S')
output_path = "../export_output_files/TXT/"
output_filename = "databricks_valut_TabDelm_" + time_string + ".txt"

In [151]:
def fetch_data_as_dict(cursor, query, params):
  try:
    cursor.execute(query, params)
    columns = [desc[0] for desc in cursor.description]
    results = cursor.fetchall()
    data_dict_list = [dict(zip(columns, row)) for row in results]
    return data_dict_list
  except sqlite3.Error as e:
    print(f"Error: {e}")
    return []

In [152]:
target_server_name = ''
target_table_name = ''
key_column_name = ''
target_column_array = [

]

In [153]:
# Get column list for the HashDiff field
query = '''
SELECT
    "''' + target_server_name + '''" AS SERVER_NAME_SOURCE
  , NULL AS DATABASE_NAME_SOURCE
  , NULL AS SCHEMA_NAME_SOURCE
  , "''' + target_table_name + '''" AS TABLE_NAME_SOURCE
  , COLUMN_NAME AS COLUMN_NAME_SOURCE
  , 'poc' AS SERVER_NAME_TARGET
  , 'pocipc' AS DATABASE_NAME_TARGET
  , 'ref_ipc' AS SCHEMA_NAME_TARGET
  , 's_''' + target_table_name + '''' AS TABLE_NAME_TARGET
  , 'HashDiff' AS COLUMN_NAME_TARGET
  , 5 AS ORDINAL_POSITION
  , 'NO' AS IS_NULLABLE
  , 'STRING' AS DATA_TYPE
  , NULL AS TRANSFORM_LOGIC
  , NULL AS TRANSFORM_GROUP
  , NULL AS TRANSFORM_GROUP_SUFFIX
  , NULL AS TRANSFORM_GROUP_LOGIC
FROM out_DataversePhysicalAttributeOutputForDataBricks
WHERE
  SERVER_NAME = ?
  AND TABLE_NAME = ?
  AND COLUMN_NAME IN ({})
'''.format(
  ",".join("?" for _ in target_column_array)
)
params = [target_server_name] + [target_table_name] + target_column_array
hash_dict_list = fetch_data_as_dict(cur, query, params)

#for row_dict in hash_dict_list:
#  print(row_dict)


In [154]:
query = '''
SELECT
    "''' + target_server_name + '''" AS SERVER_NAME_SOURCE
  , NULL AS DATABASE_NAME_SOURCE
  , NULL AS SCHEMA_NAME_SOURCE
  , "''' + target_table_name + '''" AS TABLE_NAME_SOURCE
  , COLUMN_NAME AS COLUMN_NAME_SOURCE
  , 'poc' AS SERVER_NAME_TARGET
  , 'pocipc' AS DATABASE_NAME_TARGET
  , 'ref_ipc' AS SCHEMA_NAME_TARGET
  , 's_''' + target_table_name + '''' AS TABLE_NAME_TARGET
  , COLUMN_NAME AS COLUMN_NAME_TARGET
  , ROW_NUMBER() OVER (
    ORDER BY COLUMN_NAME ASC
  ) + 5 AS ORDINAL_POSITION
  , IS_NULLABLE
  , DATA_TYPE
  , NULL AS TRANSFORM_LOGIC
  , NULL AS TRANSFORM_GROUP
  , NULL AS TRANSFORM_GROUP_SUFFIX
  , NULL AS TRANSFORM_GROUP_LOGIC
FROM out_DataversePhysicalAttributeOutputForDataBricks
WHERE
  SERVER_NAME = ?
  AND TABLE_NAME = ?
  AND COLUMN_NAME IN ({})
'''.format(
  ",".join("?" for _ in target_column_array)
)
params = [target_server_name] + [target_table_name] + target_column_array
result_dict_list = fetch_data_as_dict(cur, query, params)

#for row_dict in result_dict_list:
#  print(row_dict)


In [155]:
# Set dict list of hub
hub_fields = [
  {
      'SERVER_NAME_SOURCE': target_server_name
    , 'DATABASE_NAME_SOURCE': None
    , 'SCHEMA_NAME_SOURCE': None
    , 'TABLE_NAME_SOURCE': target_table_name
    , 'COLUMN_NAME_SOURCE': key_column_name
    , 'SERVER_NAME_TARGET': 'poc'
    , 'DATABASE_NAME_TARGET': 'pocipc'
    , 'SCHEMA_NAME_TARGET': 'ref_ipc'
    , 'TABLE_NAME_TARGET': 'h_' + target_table_name
    , 'COLUMN_NAME_TARGET': target_table_name + 'HashKey'
    , 'ORDINAL_POSITION': 1
    , 'IS_NULLABLE': 'NO'
    , 'DATA_TYPE': 'STRING'
    , 'TRANSFORM_LOGIC': 'HASH(' + target_server_name + '.' + target_table_name + '.' + key_column_name + ')'
    , 'TRANSFORM_GROUP': None
    , 'TRANSFORM_GROUP_SUFFIX': None
    , 'TRANSFORM_GROUP_LOGIC': None
  }
  , {
      'SERVER_NAME_SOURCE': None
    , 'DATABASE_NAME_SOURCE': None
    , 'SCHEMA_NAME_SOURCE': None
    , 'TABLE_NAME_SOURCE': None
    , 'COLUMN_NAME_SOURCE': None
    , 'SERVER_NAME_TARGET': 'poc'
    , 'DATABASE_NAME_TARGET': 'pocipc'
    , 'SCHEMA_NAME_TARGET': 'ref_ipc'
    , 'TABLE_NAME_TARGET': 'h_' + target_table_name
    , 'COLUMN_NAME_TARGET': 'LoadDate'
    , 'ORDINAL_POSITION': 2
    , 'IS_NULLABLE': 'NO'
    , 'DATA_TYPE': 'TIMESTAMP_NTZ'
    , 'TRANSFORM_LOGIC': 'DV load datetime'
    , 'TRANSFORM_GROUP': None
    , 'TRANSFORM_GROUP_SUFFIX': None
    , 'TRANSFORM_GROUP_LOGIC': None
  }
  , {
      'SERVER_NAME_SOURCE': None
    , 'DATABASE_NAME_SOURCE': None
    , 'SCHEMA_NAME_SOURCE': None
    , 'TABLE_NAME_SOURCE': None
    , 'COLUMN_NAME_SOURCE': None
    , 'SERVER_NAME_TARGET': 'poc'
    , 'DATABASE_NAME_TARGET': 'pocipc'
    , 'SCHEMA_NAME_TARGET': 'ref_ipc'
    , 'TABLE_NAME_TARGET': 'h_' + target_table_name
    , 'COLUMN_NAME_TARGET': 'RecordSource'
    , 'ORDINAL_POSITION': 3
    , 'IS_NULLABLE': 'NO'
    , 'DATA_TYPE': 'STRING'
    , 'TRANSFORM_LOGIC': "'" + target_server_name + '.' + target_table_name + '.' + key_column_name + "'"
    , 'TRANSFORM_GROUP': None
    , 'TRANSFORM_GROUP_SUFFIX': None
    , 'TRANSFORM_GROUP_LOGIC': None
  }
  , {
      'SERVER_NAME_SOURCE': None
    , 'DATABASE_NAME_SOURCE': None
    , 'SCHEMA_NAME_SOURCE': None
    , 'TABLE_NAME_SOURCE': None
    , 'COLUMN_NAME_SOURCE': None
    , 'SERVER_NAME_TARGET': 'poc'
    , 'DATABASE_NAME_TARGET': 'pocipc'
    , 'SCHEMA_NAME_TARGET': 'ref_ipc'
    , 'TABLE_NAME_TARGET': 'h_' + target_table_name
    , 'COLUMN_NAME_TARGET': 'ProcessID'
    , 'ORDINAL_POSITION': 4
    , 'IS_NULLABLE': 'NO'
    , 'DATA_TYPE': 'STRING'
    , 'TRANSFORM_LOGIC': 'Adatis Process ID'
    , 'TRANSFORM_GROUP': None
    , 'TRANSFORM_GROUP_SUFFIX': None
    , 'TRANSFORM_GROUP_LOGIC': None
  }
  , {
      'SERVER_NAME_SOURCE': target_server_name
    , 'DATABASE_NAME_SOURCE': None
    , 'SCHEMA_NAME_SOURCE': None
    , 'TABLE_NAME_SOURCE': target_table_name
    , 'COLUMN_NAME_SOURCE': key_column_name
    , 'SERVER_NAME_TARGET': 'poc'
    , 'DATABASE_NAME_TARGET': 'pocipc'
    , 'SCHEMA_NAME_TARGET': 'ref_ipc'
    , 'TABLE_NAME_TARGET': 'h_' + target_table_name
    , 'COLUMN_NAME_TARGET': target_table_name + 'GUID'
    , 'ORDINAL_POSITION': 5
    , 'IS_NULLABLE': 'NO'
    , 'DATA_TYPE': 'STRING'
    , 'TRANSFORM_LOGIC': None
    , 'TRANSFORM_GROUP': None
    , 'TRANSFORM_GROUP_SUFFIX': None
    , 'TRANSFORM_GROUP_LOGIC': None
  }
]

# Set dict list of sat core fields
sat_fields = [
  {
      'SERVER_NAME_SOURCE': target_server_name
    , 'DATABASE_NAME_SOURCE': None
    , 'SCHEMA_NAME_SOURCE': None
    , 'TABLE_NAME_SOURCE': target_table_name
    , 'COLUMN_NAME_SOURCE': key_column_name
    , 'SERVER_NAME_TARGET': 'poc'
    , 'DATABASE_NAME_TARGET': 'pocipc'
    , 'SCHEMA_NAME_TARGET': 'ref_ipc'
    , 'TABLE_NAME_TARGET': 's_' + target_table_name
    , 'COLUMN_NAME_TARGET': target_table_name + 'HashKey'
    , 'ORDINAL_POSITION': 1
    , 'IS_NULLABLE': 'NO'
    , 'DATA_TYPE': 'STRING'
    , 'TRANSFORM_LOGIC': 'HASH(' + target_server_name + '.' + target_table_name + '.' + key_column_name + ')'
    , 'TRANSFORM_GROUP': None
    , 'TRANSFORM_GROUP_SUFFIX': None
    , 'TRANSFORM_GROUP_LOGIC': None
  }
  , {
      'SERVER_NAME_SOURCE': None
    , 'DATABASE_NAME_SOURCE': None
    , 'SCHEMA_NAME_SOURCE': None
    , 'TABLE_NAME_SOURCE': None
    , 'COLUMN_NAME_SOURCE': None
    , 'SERVER_NAME_TARGET': 'poc'
    , 'DATABASE_NAME_TARGET': 'pocipc'
    , 'SCHEMA_NAME_TARGET': 'ref_ipc'
    , 'TABLE_NAME_TARGET': 's_' + target_table_name
    , 'COLUMN_NAME_TARGET': 'LoadDate'
    , 'ORDINAL_POSITION': 2
    , 'IS_NULLABLE': 'NO'
    , 'DATA_TYPE': 'TIMESTAMP_NTZ'
    , 'TRANSFORM_LOGIC': 'DV load datetime'
    , 'TRANSFORM_GROUP': None
    , 'TRANSFORM_GROUP_SUFFIX': None
    , 'TRANSFORM_GROUP_LOGIC': None
  }
  , {
      'SERVER_NAME_SOURCE': None
    , 'DATABASE_NAME_SOURCE': None
    , 'SCHEMA_NAME_SOURCE': None
    , 'TABLE_NAME_SOURCE': None
    , 'COLUMN_NAME_SOURCE': None
    , 'SERVER_NAME_TARGET': 'poc'
    , 'DATABASE_NAME_TARGET': 'pocipc'
    , 'SCHEMA_NAME_TARGET': 'ref_ipc'
    , 'TABLE_NAME_TARGET': 's_' + target_table_name
    , 'COLUMN_NAME_TARGET': 'RecordSource'
    , 'ORDINAL_POSITION': 3
    , 'IS_NULLABLE': 'NO'
    , 'DATA_TYPE': 'STRING'
    , 'TRANSFORM_LOGIC': "'" + target_server_name + '.' + target_table_name + "'"
    , 'TRANSFORM_GROUP': None
    , 'TRANSFORM_GROUP_SUFFIX': None
    , 'TRANSFORM_GROUP_LOGIC': None
  }
  , {
      'SERVER_NAME_SOURCE': None
    , 'DATABASE_NAME_SOURCE': None
    , 'SCHEMA_NAME_SOURCE': None
    , 'TABLE_NAME_SOURCE': None
    , 'COLUMN_NAME_SOURCE': None
    , 'SERVER_NAME_TARGET': 'poc'
    , 'DATABASE_NAME_TARGET': 'pocipc'
    , 'SCHEMA_NAME_TARGET': 'ref_ipc'
    , 'TABLE_NAME_TARGET': 's_' + target_table_name
    , 'COLUMN_NAME_TARGET': 'ProcessID'
    , 'ORDINAL_POSITION': 4
    , 'IS_NULLABLE': 'NO'
    , 'DATA_TYPE': 'STRING'
    , 'TRANSFORM_LOGIC': 'Adatis Process ID'
    , 'TRANSFORM_GROUP': None
    , 'TRANSFORM_GROUP_SUFFIX': None
    , 'TRANSFORM_GROUP_LOGIC': None
  }
]

# Append the lists
result_dict_list = hub_fields + sat_fields + hash_dict_list + result_dict_list


#for row_dict in result_dict_list:
#  print(row_dict)

In [156]:
# Write to a tab-separated text file
with open(os.path.join(output_path, output_filename), "w") as f:
  for row_dict in result_dict_list:
    line = "\t".join(str(row_dict[key]) for key in row_dict)
    f.write(line + "\n")

